In [82]:
import pandas as pd
train_dataset = pd.read_csv('training-set.csv/training-set.csv',delimiter=',')
judge_dataset = pd.read_csv('judge-set.csv/judge-set.csv',delimiter=',')


In [83]:
train_dataset.head()

,Tick#,offsetTime,IMU_ATTI(0):numSats,IMU_ATTI(0):barometer:Raw,IMU_ATTI(0):barometer:Smooth,IMU_ATTI(0):accel:X,IMU_ATTI(0):accel:Y,IMU_ATTI(0):accel:Z,IMU_ATTI(0):accel:Composite,IMU_ATTI(0):gyro:X,...,MotorCtrl:PWM:RFront,MotorCtrl:PWM:LFront,MotorCtrl:PWM:LBack,MotorCtrl:PWM:RBack,AirCraftCondition:fsmState,AirCraftCondition:nearGround,AirCraftCondition:landState,AirCraftCondition:UP_acc_t,AirCraftCondition:UP_TF_t,Operating_Phase
0,339062105,0.000,14,1188.3093,1188.0585,-0.007624,-0.162505,-0.980750,0.994151,-0.013848,...,NaN,NaN,NaN,NaN,0,True,1,0.0,0,Resting
1,339314363,0.066,14,1188.1595,1188.0468,-0.007572,-0.161164,-0.981163,0.994339,0.049079,...,NaN,NaN,NaN,NaN,0,True,1,0.0,0,Resting
2,339465911,0.105,14,1188.1595,1188.0468,-0.007572,-0.161164,-0.981163,0.994339,0.049079,...,NaN,NaN,NaN,NaN,0,True,1,0.0,0,Resting
3,339648466,0.152,14,1188.2122,1188.0359,-0.008646,-0.153160,-0.982913,0.994812,0.535372,...,NaN,NaN,NaN,NaN,1,True,1,0.0,0,Resting
4,339821470,0.197,14,1188.2122,1188.0359,-0.008646,-0.153160,-0.982913,0.994812,0.535372,...,NaN,NaN,NaN,NaN,1,True,1,0.0,0,Resting


In [84]:
total_number_of_rows = train_dataset.shape[0] 
total_number_of_columns = train_dataset.shape[1]
number_of_columns_non_numeric = len(train_dataset.select_dtypes(exclude=['number']).columns)
missing_values = train_dataset.isnull().sum()
missing_columns = [(column, missing_number) for column, missing_number in missing_values.items() if missing_number>0]
missing_columns.sort(key=lambda x: x[1], reverse=True)
summarizing =   f"total number of rows = {total_number_of_rows}\n"\
                f"total number of columns = {total_number_of_columns}\n"\
                f"number of columns having non-numeric values = {number_of_columns_non_numeric}\n"\
                f"columns with missing values = {missing_columns}\n"
print(summarizing)

total number of rows = 5085
total number of columns = 142
number of columns having non-numeric values = 5
columns with missing values = [('General:absoluteHeight', 5085), ('OA:frontDistance', 224), ('IMU_ATTI(1):directionOfTravel[true]', 32), ('IMU_ATTI(0):directionOfTravel[mag]', 8), ('IMU_ATTI(0):directionOfTravel[true]', 8), ('MotorCtrl:PWM:RFront', 5), ('MotorCtrl:PWM:LFront', 5), ('MotorCtrl:PWM:LBack', 5), ('MotorCtrl:PWM:RBack', 5), ('General:relativeHeight', 1)]



In [85]:
column_name = 'Operating_Phase'
X = train_dataset.drop(columns=[column_name])
y = train_dataset[column_name]

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12346)
# Define the list of columns to extract
columns_to_drop = [
    'General:absoluteHeight',
    'GPS(0):Date',
    'GPS(0):Time',
    'GPS:dateTimeStamp',
    'IMUEX(0):err',
    'IMUEX(1):err',
    'IMUEX(0):rtk_Alti',
    'IMUEX(1):rtk_Alti'
]

X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)


In [87]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
categorical_columns = X_train.select_dtypes(exclude=['number']).columns
X_train_ohe = pd.get_dummies(X_train,  columns=categorical_columns, dtype=int)
X_test_ohe = pd.get_dummies(X_test,  columns=categorical_columns, dtype=int)
print(categorical_columns)

Index(['AirCraftCondition:nearGround'], dtype='object')


In [88]:
missing_values = X_train_ohe.isnull().sum()
missing_counts = pd.DataFrame({'variable_name': missing_values.index, 'num_of_missing_values': missing_values.values})
X_train_ohe_imputed = X_train_ohe.fillna(X_train_ohe.mean())
X_test_ohe_imputed = X_test_ohe.fillna(X_test_ohe.mean())

In [89]:

missing_values = X_train_ohe_imputed.isnull().sum()
missing_columns = [(column, missing_number) for column, missing_number in missing_values.items() if missing_number>0]
print(missing_columns, X_train_ohe_imputed.shape[1])

[] 134


In [90]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)
correlation_matrix = X_train_ohe_imputed.corrwith(pd.Series(y_train_encoded))

correlation_matrix.dropna(inplace=True)

top75_corr = correlation_matrix.abs()#.nlargest(70)
top75_df = pd.DataFrame({'variable': top75_corr.index, 'corr_score': top75_corr.values})
print(top75_df)


c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


                               variable  corr_score
0                                 Tick#    0.004182
1                            offsetTime    0.004182
2                   IMU_ATTI(0):numSats    0.015660
3             IMU_ATTI(0):barometer:Raw    0.022762
4          IMU_ATTI(0):barometer:Smooth    0.020224
..                                  ...         ...
116          AirCraftCondition:fsmState    0.009518
117         AirCraftCondition:landState    0.000189
118          AirCraftCondition:UP_acc_t    0.008056
119  AirCraftCondition:nearGround_False    0.017976
120   AirCraftCondition:nearGround_True    0.017976

[121 rows x 2 columns]


In [91]:
y_train_encoded

array([0, 0, 0, ..., 1, 0, 0])

In [92]:
top75_variables = top75_df['variable'].tolist()

X_train_t75 = X_train_ohe_imputed[top75_variables]
X_test_t75 = X_test_ohe_imputed[top75_variables]


In [93]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled_mm = scaler.fit_transform(X_train_t75)
X_test_scaled_mm = scaler.fit_transform(X_test_t75)

In [94]:
from sklearn.preprocessing import StandardScaler
scaler_s = StandardScaler()
X_train_scaled_std = scaler_s.fit_transform(X_train_t75)
X_test_scaled_std = scaler_s.fit_transform(X_test_t75)
print(X_train_scaled_std,X_test_scaled_std)

[[-0.12412592 -0.12411939  0.06040737 ... -0.55339394  0.50221186
  -0.50221186]
 [ 0.07727642  0.07728184  0.06040737 ... -0.55339394  0.50221186
  -0.50221186]
 [ 0.07433918  0.07433592  0.06040737 ... -0.55339394  0.50221186
  -0.50221186]
 ...
 [-0.52344046 -0.52343653  1.54642876 ... -0.55339394  0.50221186
  -0.50221186]
 [ 1.51458441  1.51458566  0.06040737 ... -0.55339394  0.50221186
  -0.50221186]
 [-1.18635037 -1.18635112  0.06040737 ... -0.55339394  0.50221186
  -0.50221186]] [[ 0.0030501   0.00305428 -0.01674033 ... -0.56504188  0.48348075
  -0.48348075]
 [ 1.71629295  1.7162896  -1.53771876 ... -0.56504188 -2.06833467
   2.06833467]
 [-0.10959081 -0.10959604 -0.01674033 ... -0.56504188  0.48348075
  -0.48348075]
 ...
 [ 1.56802141  1.5680248  -0.01674033 ... -0.23564991  0.48348075
  -0.48348075]
 [ 0.28728041  0.28728122 -0.01674033 ... -0.56504188  0.48348075
  -0.48348075]
 [-1.13002836 -1.13002909 -0.01674033 ... -0.56504188  0.48348075
  -0.48348075]]


In [95]:
"""from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the Random Forest classifier on the training data
rf_classifier.fit(X_train_t75, y_train_encoded)

# Predict on the test data
y_pred = rf_classifier.predict(X_test_t75)

# Calculate and print the accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
"""
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Create an XGBoost classifier
xgb_classifier = xgb.XGBClassifier(n_estimators=100, random_state=42)

X_train_t75.columns = [col.replace('[', '_').replace(']', '_') for col in X_train_t75.columns]
X_test_t75.columns = [col.replace('[', '_').replace(']', '_') for col in X_test_t75.columns]

# Fit the XGBoost classifier on the training data
xgb_classifier.fit(X_train_t75, y_train_encoded)

# Predict on the test data
y_pred = xgb_classifier.predict(X_test_t75)




c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:359: FutureWarn

In [96]:
# Calculate and print the accuracy
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 99.69%


In [97]:
from sklearn.metrics import accuracy_score, precision_score

# Calculate accuracy
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Calculate precision
precision = precision_score(y_test_encoded, y_pred, average='macro')  # You can use 'micro' or 'weighted' as well
print(f"Test Precision: {precision * 100:.2f}%")


Test Accuracy: 99.69%
Test Precision: 99.85%


In [98]:
judge_data = judge_dataset.drop(columns=columns_to_drop)

In [99]:
categorical_columns = judge_data.select_dtypes(exclude=['number']).columns
judge_data_ohe = pd.get_dummies(judge_data,  columns=categorical_columns, dtype=int)
missing_values = judge_data_ohe.isnull().sum()
missing_counts = pd.DataFrame({'variable_name': missing_values.index, 'num_of_missing_values': missing_values.values})
judge_data_ohe_imputed = X_train_ohe.fillna(X_train_ohe.mean())
print(judge_data_ohe.columns)
judge_data_ohe['AirCraftCondition:nearGround_False'] = 0
print(judge_data_ohe.head())

Index(['Tick#', 'offsetTime', 'IMU_ATTI(0):numSats',
       'IMU_ATTI(0):barometer:Raw', 'IMU_ATTI(0):barometer:Smooth',
       'IMU_ATTI(0):accel:X', 'IMU_ATTI(0):accel:Y', 'IMU_ATTI(0):accel:Z',
       'IMU_ATTI(0):accel:Composite', 'IMU_ATTI(0):gyro:X',
       ...
       'IMU_ATTI(1):temperature', 'MotorCtrl:PWM:RFront',
       'MotorCtrl:PWM:LFront', 'MotorCtrl:PWM:LBack', 'MotorCtrl:PWM:RBack',
       'AirCraftCondition:fsmState', 'AirCraftCondition:landState',
       'AirCraftCondition:UP_acc_t', 'AirCraftCondition:UP_TF_t',
       'AirCraftCondition:nearGround_True'],
      dtype='object', length=133)
       Tick#  offsetTime  IMU_ATTI(0):numSats  IMU_ATTI(0):barometer:Raw  \
0  143222258       0.000                   11                   1230.502   
1  143393138       0.044                   11                   1230.502   
2  143566452       0.089                   11                   1230.404   
3  143758752       0.139                   11                   1230.404   
4  1

In [100]:
judge_data_t75 = judge_data_ohe[top75_variables]
X_judge_scaled_std = scaler_s.fit_transform(judge_data_t75)
 

In [101]:
"""model.fit(X_train_t75, y_train_encoded, epochs=10, batch_size=32)

y_test_encoded = to_categorical(y_test_encoded, num_classes=num_classes)
test_loss, test_accuracy = model.evaluate(X_test_t75, y_test_encoded)
"""

'model.fit(X_train_t75, y_train_encoded, epochs=10, batch_size=32)\n\ny_test_encoded = to_categorical(y_test_encoded, num_classes=num_classes)\ntest_loss, test_accuracy = model.evaluate(X_test_t75, y_test_encoded)\n'

In [102]:
"""from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the Random Forest classifier on the training data
rf_classifier.fit(X_train_t75, y_train_encoded)

"""
import xgboost as xgb

# Create an XGBoost classifier
xgb_classifier = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Fit the XGBoost classifier on the training data
xgb_classifier.fit(X_train_t75, y_train_encoded)

# Predict on the test data
y_pred = xgb_classifier.predict(X_test_t75)


c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:359: FutureWarn

In [104]:
"""from sklearn.impute import SimpleImputer

# Create an imputer that replaces NaN values with the mean of each column
imputer = SimpleImputer(strategy='mean')

# Fit the imputer on your judge dataset and transform it
judge_data_imputed = imputer.fit_transform(judge_data_t75)

# Create a new DataFrame with the imputed values
judge_data_imputed_df = pd.DataFrame(judge_data_imputed, columns=judge_data_t75.columns)

# Now, you can use this imputed dataset for prediction
predicted_operating_phase = rf_classifier.predict(judge_data_imputed_df)
"""
import xgboost as xgb
from sklearn.impute import SimpleImputer

# Create an imputer that replaces NaN values with the mean of each column
imputer = SimpleImputer(strategy='mean')

# Fit the imputer on your judge dataset and transform it
judge_data_imputed = imputer.fit_transform(judge_data_t75)

# Create a new DataFrame with the imputed values
judge_data_imputed_df = pd.DataFrame(judge_data_imputed, columns=judge_data_t75.columns)

predicted_operating_phase = xgb_classifier.predict(judge_data_imputed_df)



c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\vrjav\Downloads\Assignments\Fall 2023\Machine Learning\assignment\Lib\site-packages\xgboost\data.py:359: FutureWarn

ValueError: feature_names mismatch: ['Tick#', 'offsetTime', 'IMU_ATTI(0):numSats', 'IMU_ATTI(0):barometer:Raw', 'IMU_ATTI(0):barometer:Smooth', 'IMU_ATTI(0):accel:X', 'IMU_ATTI(0):accel:Y', 'IMU_ATTI(0):accel:Z', 'IMU_ATTI(0):accel:Composite', 'IMU_ATTI(0):gyro:X', 'IMU_ATTI(0):gyro:Y', 'IMU_ATTI(0):gyro:Z', 'IMU_ATTI(0):gyro:Composite', 'IMU_ATTI(0):mag:X', 'IMU_ATTI(0):mag:Y', 'IMU_ATTI(0):mag:Z', 'IMU_ATTI(0):mag:Mod', 'IMU_ATTI(0):velN', 'IMU_ATTI(0):velE', 'IMU_ATTI(0):velD', 'IMU_ATTI(0):velComposite', 'IMU_ATTI(0):velH', 'IMU_ATTI(0):GPS-H', 'IMU_ATTI(0):roll', 'IMU_ATTI(0):pitch', 'IMU_ATTI(0):yaw', 'IMU_ATTI(0):yaw360', 'IMU_ATTI(0):totalGyro:Z', 'IMU_ATTI(0):totalGyro:X', 'IMU_ATTI(0):totalGyro:Y', 'IMU_ATTI(0):magYaw', 'IMU_ATTI(0):directionOfTravel_mag_', 'IMU_ATTI(0):directionOfTravel_true_', 'IMU_ATTI(0):temperature', 'General:relativeHeight', 'GPS(0):heightMSL', 'GPS(0):hDOP', 'GPS(0):pDOP', 'GPS(0):sAcc', 'GPS(0):numGPS', 'GPS(0):numGLNAS', 'GPS(0):numSV', 'GPS(0):velN'

In [51]:

# Map the numerical predictions back to their corresponding labels
class_labels = ['Resting', 'Taking-Off', 'Hovering', 'Climbing', 'Landing']
predicted_operating_phase_labels = [class_labels[prediction] for prediction in predicted_operating_phase]

# Create a new dataset containing "Tick#" and "Operating_Phase" columns
new_dataset = judge_data[["Tick#"]].copy()
new_dataset["Operating_Phase"] = predicted_operating_phase_labels

# Save the new dataset to a CSV file
new_dataset.to_csv("new_dataset_with_tick_and_operating_phase_1.csv", index=False)

print("Prediction and CSV export completed.")

Prediction and CSV export completed.
